In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
gene_list, alel_list, abun_list = [], [], []
def read_and_filter(file, threshold=10):
    with open(file) as infile:
        for line in infile:
            if 'abundance' in line:
                gene = line.strip().split(' ')[2].split('*')[0]
                alel = line.strip().split(' ')[2].split('*')[1]
                abun = float(line.strip().split(' ')[4].split('%')[0])
                
                if abun >= threshold:
                    gene_list.append(gene)
                    alel_list.append(alel)
                    abun_list.append(abun)
                    
    gene_alel_abun_dict = {'Gene':gene_list, 'Variant':alel_list, 'Abundance':abun_list}
    
    filtered_df = pd.DataFrame.from_dict(gene_alel_abun_dict)
    
    return filtered_df

In [3]:
df = read_and_filter('NA12877.hla.report.txt')

In [4]:
df

,Gene,Variant,Abundance
0,A,03:01:01:01,29.54
1,A,02:01:01:01,28.58
2,A,03:21N,18.14
3,A,03:78,18.14
4,B,44:02:01:01,28.93
...,...,...,...
59,TAP1,06:01,40.53
60,TAP1,01:02N,13.53
61,TAP2,01:04,69.84
62,TAP2,01:01:02,30.16


In [5]:
def write_2_excel(df, filename, genes='All'):
    if genes == 'All':
        df.to_excel(filename[:-4]+'.xlsx', index=False)
    else:
        df_subset = df[df['Gene'].isin(genes)]
        df_subset.to_excel(filename[:-4]+'_subset.xlsx', index=False)

In [6]:
write_2_excel(df, 'NA12877.hla.report.txt')

In [8]:
def plot_variants(df, gene='All'):
    
    df['Gene_Var'] = df.Gene + ':' + df.Variant
    
    if gene == 'All':
        sns.set(rc={'figure.figsize':(11.7,11.27)})
        sns.barplot(data = df, x = 'Abundance', y = 'Gene_Var')
        
    else:
        df_subset = df[df['Gene'] == gene]
        plt.pie(df_subset.Abundance, labels=df_subset.Variant, autopct='%.0f%%')
        plt.title(f"Variant abundances for gene {gene}")

In [11]:
def hisat_genotype_analyzer(file, mode, threshold=10, genes='All'):
    """
    param 
    """
    
    hisat_genotype_df = read_and_filter(file, threshold)
    
    if mode == 'write' or mode == 'excel':
        write_2_excel(hisat_genotype_df, file, genes)
        
    elif mode == 'plot' or mode == 'figure' or mode == 'fig':
        plot_variants(hisat_genotype_df, genes)
        
    else:
        print(f'I can not do {mode}')
        print('Chose one of the existing modes: wrte or plot')

In [12]:
hisat_genotype_analyzer('NA12877.hla.report.txt', 'excel', genes=['A','B'])